### interspecific calculations
- calculate pairwise distances between all species in list a to all in list b and append to list_inter and list_inter_speciesab
- Append smallest distance from list_inter_speciesab to list_Dmin_NN
- iterate over all species list combinations
- export list_inter and list_Dmin_NN to csv

- use itertools: https://stackoverflow.com/questions/31206106/compare-each-item-of-two-lists-in-python
- not going to work I think: https://stackoverflow.com/questions/28120527/python-pairwise-comparison-between-two-lists-list-a-list-b


In [1]:
import numpy as np
import pandas as pd
from Bio.Alphabet import IUPAC, Gapped
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Biopython

basic syntax:
- sequence = record.seq
- id = record.id

aln = AlignIO.read(open('test.fas'), 'fasta')

print(aln)

## IUPAC compatible p-distance calculator function [WORKS]

Ignore IUPAC codes in comparison of two sequences (as strings)

In [13]:
def IUPACdistance(seq1, seq2):
    ignorelist = ["N","?","-","M","R","W","S","Y","K","V","H","D","B"]
    unamblengthseq1 = len(seq1.translate(str.maketrans('','','N?-MRWSYKVHDB')))
    unamblengthseq2 = len(seq2.translate(str.maketrans('','','N?-MRWSYKVHDB')))
    comparedlength = min(unamblengthseq1, unamblengthseq2)
    difference = 0
    for x, y in zip(seq1.upper(), seq2.upper()):
        if x in ignorelist or y in ignorelist:
            difference += 0
        elif x != y:
            difference += 1
    dpairwise = (difference / comparedlength)
    return dpairwise

seq1 = 'AARCTGACACT'
seq2 = 'TRCCTGN-ACT'

IUPACdistance(seq1, seq2)

0.125

## list of species function [WORKS]

In [10]:
def createlistofspecies(inputfile, fileformat):
    sequences = SeqIO.parse(inputfile, fileformat, alphabet=IUPAC.ambiguous_dna)
    listofspecies = []
    for record in sequences:
        speciesname = (record.id.split(".")[1])
        if speciesname not in listofspecies:
            listofspecies.append(speciesname)
    print(str(len(listofspecies)) + " species in charactermatrix.")
    return listofspecies

createlistofspecies('test.fas','fasta')

30 species in charactermatrix.


['Phyllonorycter_issikii',
 'Phyllonorycter_ulmifoliella',
 'Phyllonorycter_spinicolella',
 'Cameraria_ohridella',
 'Caloptilia_staintoni',
 'Phyllonorycter_corylifoliella',
 'Caloptilia_braccatella',
 'Caloptilia_flava',
 'Povolnya_leucapennella',
 'Aristaea_pavoniella',
 'Phyllonorycter_kuhlweiniella',
 'Phyllonorycter_muelleriella',
 'Phyllonorycter_acerifoliella',
 'Triberta_helianthemella',
 'Phyllonorycter_alpina',
 'Phyllonorycter_AlnuscordataItaly',
 'Phyllocnistis_xenia',
 'Parornix_torquillella',
 'Phyllonorycter_abrasella',
 'Caloptilia_cuculipennella',
 'Aspilapteryx_tringipennella',
 'Leucospilapteryx_omissella',
 'Dialectica_imperialella',
 'Caloptilia_elongella',
 'Phyllonorycter_hilarella',
 'Phyllocnistis_unipunctella',
 'Phyllonorycter_trifasciella',
 'Parornix_polygrammella',
 'Phyllonorycter_nicellii',
 'Parornix_loganella']

## list of records per species snippet [WORKS]

In [116]:
for speciesname in listofspecies:
    print(speciesname)
    speciessamples = []
    sequences = SeqIO.parse('test.fas', 'fasta', alphabet=IUPAC.ambiguous_dna)
    for record in sequences:
        if (record.id.split(".")[1]) == speciesname:
            speciessamples.append(record)
    print(speciessamples)

Phyllonorycter_issikii
[SeqRecord(seq=Seq('ACTCTCTATTTTATTTTTGGAATCTGAGCAGGAATAATTGGATCTTCTTTAAGA...NNN', IUPACAmbiguousDNA()), id='ISSIK249N14.Phyllonorycter_issikii', name='ISSIK249N14.Phyllonorycter_issikii', description='ISSIK249N14.Phyllonorycter_issikii', dbxrefs=[])]
Phyllonorycter_ulmifoliella
[SeqRecord(seq=Seq('ACTCTTTATTTTATTTTTGGAATTTGAGCAGGAATAATTGGATCTTCTTTAAGA...TTA', IUPACAmbiguousDNA()), id='LNAUV1219N17.Phyllonorycter_ulmifoliella', name='LNAUV1219N17.Phyllonorycter_ulmifoliella', description='LNAUV1219N17.Phyllonorycter_ulmifoliella', dbxrefs=[])]
Phyllonorycter_spinicolella
[SeqRecord(seq=Seq('ACTCTTTATTTTATTTTTGGAATTTGAGCAGGAATAATTGGATCCTCTTTAAGA...TTA', IUPACAmbiguousDNA()), id='LNAUV1218N17.Phyllonorycter_spinicolella', name='LNAUV1218N17.Phyllonorycter_spinicolella', description='LNAUV1218N17.Phyllonorycter_spinicolella', dbxrefs=[])]
Cameraria_ohridella
[SeqRecord(seq=Seq('ACTTTATATTTTATTTTTGGAATTTGAGCAGGAATAGTTGGTTCATCTTTAAGA...TTA', IUPACAmbiguousDNA()), id='

## Pairwise distances within species [WORKS]

- Outputs Dmax
- Outputs all instraspecific values

In [24]:
inputfile = 'test.fas'
dmax_outputfile = 'dmax.csv'
speciesintra_outputfile = 'species_intra.csv'

listofspecies = createlistofspecies('test.fas','fasta')

dmaxvalues = []
allintra = []
for speciesname in listofspecies:
    recordlist = []
    distancelist = []
    sequences = SeqIO.parse(inputfile, 'fasta', alphabet=IUPAC.ambiguous_dna)
    for record in sequences:
        if (record.id.split(".")[1]) == speciesname:        
            recordlist.append(record)
    if len(recordlist) > 1:
        for a, b in itertools.combinations(recordlist, 2):
            distancelist.append(IUPACdistance(str(a.seq), str(b.seq)))        
        dmaxvalues.append({'species': speciesname,
                           'n_seqs': len(recordlist),
                           'dmax': max(distancelist)})
        allintra.append(distancelist)
    else:
        dmaxvalues.append({'species': speciesname, 'n_seqs': len(recordlist)})

df_dmax = pd.DataFrame(dmaxvalues)
df_dmax
#totaldf.to_csv(dmax_outputfile, index=False)

flat_allintra = [item for sublist in allintra for item in sublist]
df_intra = pd.DataFrame(flat_allintra)
df_intra.rename(columns={0:'species_intra'}, inplace=True)
df_intra
#df_intra.to_csv(speciesintra_outputfile, index=False)

30 species in charactermatrix.


,species_intra
0,0.012389
1,0.003540
2,0.008850
3,0.057245
4,0.053667
5,0.021798
6,0.052863
7,0.012376
8,0.006116
9,0.051771


## Haplotype counter [WORKS]

## Inter pairwise comparisons [To do]

- make recordlists for all species in listofspecies
- compare all sequences against each other and for each species make dict entry for NN?

Verbose version of the function to compare sequence1 to sequence2 allowing for IUPAC ambiguities

In [14]:
def IUPACdistance_verbose(seq1, seq2):
    ignorelist = ["N","?","-","M","R","W","S","Y","K","V","H","D","B"]
    unamblengthseq1 = len(seq1.translate(str.maketrans('','','N?-MRWSYKVHDB')))
    print("seq1 bp: " + str(unamblengthseq1))
    unamblengthseq2 = len(seq2.translate(str.maketrans('','','N?-MRWSYKVHDB')))
    print("seq2 bp: " + str(unamblengthseq2))
    comparedlength = min(unamblengthseq1, unamblengthseq2)
    print("compared length: " + str(comparedlength))
    difference = 0
    for x, y in zip(seq1.upper(), seq2.upper()):
        if x in ignorelist or y in ignorelist:
            difference += 0
        elif x != y:
            difference += 1
    print("difference:" + str(difference))
    dpairwise = (difference / comparedlength)
    return dpairwise 

seq1 = 'AARCTGACACT'
seq2 = 'TRCCTGN-ACT'

IUPACdistance_verbose(seq1, seq2)

seq1 bp: 10
seq2 bp: 8
compared length: 8
difference:1


0.125